# Method 3: Neural network

In [2]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import kaleido
import os
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import numdifftools as nd
from numpy.random import uniform
import math as m
import time
from numba import jit

from scipy import stats

from sklearn import datasets
from sklearn.model_selection import train_test_split , KFold
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import warnings

from collections import Counter
# setup offline mode
py.offline.init_notebook_mode(connected=True)

pd.set_option('display.max_rows', 20)
warnings.filterwarnings('ignore')

In [3]:
CZ_ALPHABET = ['a', 'á', 'b', 'c', 'č', 'd', 'ď',
               'e', 'é', 'ě', 'f', 'g', 'h', 'ch',
               'i', 'í', 'j', 'k', 'l', 'm', 'n',
               'ň', 'o', 'ó', 'p', 'q', 'r', 'ř',
               's', 'š', 't', 'ť', 'u', 'ú','ů',
               'v', 'w', 'x', 'y', 'ý','z', 'ž', ' ']

def word_to_vector(word: str) -> np.array:
    array = np.zeros(len(CZ_ALPHABET))
    for i, alph in enumerate(CZ_ALPHABET):
        array[i] = word.count(alph)
    return pd.Series(array)

def calculate_hidden_layer_size(Ns: int, Ni: int, No: int, alpha: int) -> float:
    Nh = Ns / (alpha * (Ni - No))
    return Nh

def sigmoid(x):
    y = 1 / (1 + np.e ** (-x))
    return y

def sigmoid_derivative(x):
    y = sigmoid(x) * (1 - sigmoid(x))
    return y

def ReLU(x):
    y = x if x >= 0 else 0
    return y

In [4]:
data = pd.read_csv('data.csv', encoding = 'ansi', usecols=['Obec', 'Kraj'])
data['Obec'] = data['Obec'].str.lower()
output_dict = list(np.sort(data['Kraj'].unique()).flatten())

data[CZ_ALPHABET] = data['Obec'].apply(word_to_vector)
data['expected'] = data['Kraj'].apply(lambda x: (np.array(output_dict) == x).astype(int))

data.head()

,Obec,Kraj,a,á,b,c,č,d,ď,e,...,ů,v,w,x,y,ý,z,ž,,expected
0,abertamy,Karlovarský kraj,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,adamov,Jihočeský kraj,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,adamov,Jihomoravský kraj,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,adamov,Středočeský kraj,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,adršpach,Královéhradecký kraj,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


Using the following to get the size of the hidden layer

$$
N_\mathrm{h} = \frac{N_\mathrm{s}}{\alpha (N_\mathrm{i} + N_\mathrm{o})}
$$
- $N_\mathrm{s}$ - number of samples in training data
- $N_\mathrm{i}$ - number of input neurons (43)
- $N_\mathrm{o}$ - number of output neurons (14)
- $\alpha$ - an arbitrary number usually between 2 and 10

In [6]:
class NN:
    def __init__(self, size: np.array, func: str = 'sigmoid', x_train: np.ndarray = None,
                 y_train: np.array = None, eta: float = 0.01, backpropagation: bool = False):

        self.train = x_train
        self.expected = y_train
        self.size = size
        self.func = func
        self.eta = eta
        self.backpropagation = backpropagation
        self.layers = np.array([uniform(size=n) for n in size], dtype=object)

        self.n_layers = len(size)

        self.n_weights = np.dot(size[:-1], size[1:])
        self.n_biases = np.sum(size[1:])

        self.n_params = self.n_weights + self.n_biases
        self.params = uniform(size=self.n_params)
        # initialize weights
        self.W = None
        self.update_weights()
        # initialize biases
        self.B = None
        self.update_biases()

    def a(self, L: int, k: int):
        if L == 0:
            a = layers[0, k]
        else:
            a = sigmoid(self.z(L=L, k=k))
        return a

    def z(self, L: int, k: int):
        z = np.dot(self.W[L-1, k] * self.layers[L-1]) + self.B[L-1, k]
        return z

    def w(self, L: int, i: int, j: int):
        w = self.W[L-1][i, j]
        return w

    def b(self, L: int, k: int):
        b = self.B[L-1, k]
        return b

    def update_weights(self):
        weights = self.params[:self.n_weights]
        lengths = [self.size[i] * self.size[i+1] for i in range(self.n_layers-1)]
        slices = self.slice_from_lengths(lengths=lengths)
        weights = np.split(weights, slices)[:-1]

        for i in range(len(weights)):
            newshape = (self.size[i+1], self.size[i])
            weights[i] = weights[i].reshape(newshape)
        self.W = weights

    def update_biases(self):
        biases = self.params[:self.n_biases]
        lengths = self.size[1:]
        slices = self.slice_from_lengths(lengths=lengths)
        biases = np.split(biases, slices)
        self.B = biases

    def propagate_forward(self):
        self.update_weights()
        self.update_biases()
        f = ReLU if self.func == 'ReLU' else sigmoid
        for i in range(self.n_layers-1):
            self.layers[i+1] = f(np.dot(self.W[i], self.layers[i]) + self.B[i])

    def feed_input(self, layer0):
        self.layers[0] = layer0

    def get_output(self):
        return self.layers[-1]

    def slice_from_lengths(self, lengths):
        slices = [np.sum(lengths[:i]) for i in range(self.n_layers)][1:]
        return slices

    def predict(self, vector: np.array):
        if vector.shape[0] != self.size[0]:
            print(f'Vector shape {vector.shape} does not match layer shape {self.size[0]}')
            raise TypeError
        self.feed_input(vector)
        self.propagate_forward()
        prediction = self.get_output()
        return prediction

    def predict_word(self, word: str):
        vector = word_to_vector(word)
        return self.predict(vector)

    def cost(self, vector: np.array, expected: np.array):
        prediction = self.predict(vector)
        c = np.sum((prediction - expected) ** 2)
        return c

    def cost_of_sample(self, params: np.ndarray):
        sample_size = len(self.train)
        self.params = params
        costs = np.array([self.cost(self.train[i], self.expected[i]) for i in range(sample_size)])
        return np.mean(costs)

    def neg_grad(self):
        if self.backpropagation:
            grad = 0
        else:
            grad = nd.Gradient(self.cost_of_sample)(self.params)
        return -grad

    # functions for backpropagation
    def d_l(self, l: int):
        if l == self.n_layers - 1:
            d_l = np.multiply(self.dE_dA(l), self.dA_dZ(l))
        else:
            d_l = np.dot(self.d_l(l+1).T, self.W[l])
            d_l = np.multiply(d_l, self.dA_dZ(l))
        return d_l

    def X(self, l: int):
        return self.layers[l]

    def dE_dA(self, l: int):
        if l == self.n_layers - 1:
            pass
        else:
            pass

    def dA_dZ(self, l: int):
        pass

    def dE_dW(self, l: int):
        dE_dW = np.multiply(self.d_l(l), self.X(l-1))
        return dE_dW

    def train_network(self):
        neg_grad = self.neg_grad()
        norm_factor = self.eta * np.linalg.norm(self.params) / np.linalg.norm(neg_grad)
        self.params += norm_factor * neg_grad

    def compute_accuracy(self, x_test: np.ndarray, y_test: np.ndarray):
        pass

In [7]:
# normalizovat input!!!

x = np.array(data[CZ_ALPHABET])
y = np.array(data['expected'])

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST= train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 0)

# normalize input?
Ns = len(X_TRAIN)
Ni = len(CZ_ALPHABET)
No = len(output_dict)
ALPHA = 5
FUNC = 'sigmoid'
ETA = 0.05
N_ITER = 8

X_TRAIN

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
np.random.seed(0)

hidden_layer_size = int(np.ceil(calculate_hidden_layer_size(Ns=Ns, Ni=Ni, No=No, alpha=ALPHA)))
network_size = np.array([Ni, hidden_layer_size, No], dtype=object)


print(f'Training the neural network with a sample of {Ns} names.')
print(f'alpha = {ALPHA} → optimal hidden layer size: {hidden_layer_size}')
print(f'Initialising a neural network with shape {(Ni, hidden_layer_size, No)}')
print(f'Activation function is set to {FUNC}')
print(f'Learning rate is set to {ETA}')


network = NN(size=network_size, func=FUNC, x_train=X_TRAIN[:700], y_train=Y_TRAIN[:700], eta=ETA, backpropagation=False)

print(network.W[0].shape)
start = time.time()
print('Starting learning session')
# network.cost_of_sample(vector_sample=vector_sample, expected_sample=expected_sample)
print('Cost before', network.cost_of_sample(network.params))

for i in range(N_ITER):
    s = time.time()
    print(f'Currently at i = {i}', end='\r')
    network.train_network()
    e = time.time()
    print(f'Iteration took {e - s:.2f} seconds')

print('Cost after', network.cost_of_sample(network.params))
end = time.time()
print(f'Finished in {end-start:.2f} seconds')

# print(network.cost_of_sample(vector_sample=vector_sample, expected_sample=expected_sample))


KeyboardInterrupt: 

In [ ]:
data['prediction'] = data['Obec'].apply(lambda x: output_dict[np.argmax(network.predict_word(word=x))])
data['is_correct'] = data['Kraj'] == data['prediction']

data['is_correct'].mean()

In [ ]:
arr = np.tile([1, 2, 3], (3, 1))
arr.shape

In [ ]:
shape = [43, 25, 14]
n_weights = np.dot(shape[1:], shape[:-1])
weights = np.random.uniform(size=n_weights)

indeces = np.cumsum(np.multiply(shape[1:], shape[:-1]))

weights = np.split(weights, indeces)[:-1]
print(len(weights))
for layer in weights:
    print(layer.shape)

In [ ]:
total = 0
def row_sum(x, y):
    row_sum = x + y
    global total
    total += row_sum
    return row_sum

df = pd.DataFrame({"A": [1, 2, 3],
                   "B": [3, 4, 5]
                   })

df.apply(lambda row: row_sum(row[0], row[1]), axis=1)
print(total)

In [14]:
data.iloc[:, 1]

0           Karlovarský kraj
1             Jihočeský kraj
2          Jihomoravský kraj
3           Středočeský kraj
4       Královéhradecký kraj
                ...         
6254        Karlovarský kraj
6255          Olomoucký kraj
6256         Pardubický kraj
6257          Jihočeský kraj
6258        Středočeský kraj
Name: Kraj, Length: 6259, dtype: object

In [7]:
func_dict = {
    sigmoid: ReLU
}

func_dict[sigmoid](-5)

0

In [65]:
activities = ['kreslení', 'povídání', 'předvádění']
subjects = ['matematika', 'fyzika']
i_a = np.random.randint(0, 3)
i_s = np.random.randint(0, 2)
print(activities[i_a], subjects[i_s])

povídání matematika


In [19]:
muffin = np.random.uniform(size=43)
pizza = np.random.uniform(size=43)
kebab = np.random.uniform(size=43)
apple = np.random.uniform(size=43)

exp1, exp2, exp3 = np.random.uniform(size=13), np.random.uniform(size=13), np.random.uniform(size=13)

data = pd.DataFrame({
    'food': (muffin, pizza, kebab, apple),
    'expected': (exp1, exp2, exp3, exp3)
})



[[array([0.90775354, 0.36902822, 0.14569077, 0.17580159, 0.37288331,
         0.6992771 , 0.98591966, 0.98977379, 0.51281808, 0.02207219,
         0.83584847, 0.68306341, 0.5965872 , 0.69454343, 0.11882914,
         0.91503272, 0.38663549, 0.56692535, 0.30788714, 0.35795568,
         0.56077635, 0.99669272, 0.62804393, 0.99720054, 0.61373387,
         0.49082924, 0.67636714, 0.55717322, 0.24304509, 0.09580556,
         0.10558262, 0.57676409, 0.05534122, 0.34439121, 0.8314402 ,
         0.73389861, 0.86338317, 0.36381457, 0.07317685, 0.18910632,
         0.58109321, 0.39700182, 0.75520731]),
  array([0.45218832, 0.48980106, 0.32087086, 0.73056332, 0.53754062,
         0.20997717, 0.90594164, 0.88067984, 0.42494239, 0.17382308,
         0.41773399, 0.95456796, 0.45323998])],
 [array([0.88481045, 0.9883843 , 0.84970419, 0.56138503, 0.6599603 ,
         0.44406562, 0.08959943, 0.57333434, 0.47983568, 0.68514501,
         0.38133234, 0.92544897, 0.37488609, 0.88327933, 0.2966762 ,
        